In [1]:
import pandas as pd
import numpy as np
import nltk
import h2o
#import tensorflow
import matplotlib.pyplot as plt
import gensim.downloader as api
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim import models 
from gensim.models import KeyedVectors, Word2Vec
from sentence_transformers import SentenceTransformer
#from tensorflow.python.keras.preprocessing.text import Tokenizer

import warnings
warnings.filterwarnings('ignore')



<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
data_url = ("/Users/vidyakumar/Desktop/python/apziva/potential-talents - Aspiring human resources - seeking human resources.csv")
pd.set_option('display.max_colwidth', None)
get_ipython().run_line_magic('matplotlib', 'inline')

In [3]:
def load_data():
    data = pd.read_csv(data_url)
    return data

data = load_data()

data.head()

,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional,"Houston, Texas",85,NaN
1,2,Native English Teacher at EPIK,Kanada,500+,NaN
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN


In [4]:
# If it's the same job description in the same city, for the same job title, we consider it duplicate.
print(data.shape)
df_nodups = data.drop_duplicates(subset=['job_title', 'location'])
print(df_nodups.shape)

(104, 5)
(54, 5)


In [5]:
df_nodups.isnull().sum()

id             0
job_title      0
location       0
connection     0
fit           54
dtype: int64

In [6]:
df_nodups.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54 entries, 0 to 103
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          54 non-null     int64  
 1   job_title   54 non-null     object 
 2   location    54 non-null     object 
 3   connection  54 non-null     object 
 4   fit         0 non-null      float64
dtypes: float64(1), int64(1), object(3)
memory usage: 2.5+ KB


In [7]:
#replace acronymns
df_nodups['job_title'] = df_nodups['job_title'].str.replace("HRIS", " Human Resources Information System ")
df_nodups['job_title'] = df_nodups['job_title'].str.replace("HR", " Human Resources ")
df_nodups['job_title'] = df_nodups['job_title'].str.replace("EPIK", " English Program in Korea ")
df_nodups['job_title'] = df_nodups['job_title'].str.replace("JTI", " Japan Tobacco International ")

#removing stopwords and making it all lowercase
stop_words = set(stopwords.words('english'))
df_nodups['job_title_no_stpw'] = df_nodups['job_title'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
df_nodups['job_title_no_stpw'] = df_nodups['job_title_no_stpw'].str.lower()
df_nodups.head(10)

,id,job_title,location,connection,fit,job_title_no_stpw
0,1,2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional,"Houston, Texas",85,NaN,2019 c.t. bauer college business graduate (magna cum laude) aspiring human resources professional
1,2,Native English Teacher at English Program in Korea,Kanada,500+,NaN,native english teacher english program korea
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN,aspiring human resources professional
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN,people development coordinator ryan
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN,advisory board member celal bayar university
5,6,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN,aspiring human resources specialist
6,7,Student at Humber College and Aspiring Human Resources Generalist,Kanada,61,NaN,student humber college aspiring human resources generalist
7,8,Human Resources Senior Specialist,San Francisco Bay Area,500+,NaN,human resources senior specialist
9,10,Seeking Human Resources Human Resources Information System and Generalist Positions,Greater Philadelphia Area,500+,NaN,seeking human resources human resources information system generalist positions
10,11,Student at Chapman University,"Lake Forest, California",2,NaN,student chapman university


In [8]:
#removing digits and special characters
df_nodups['job_title_no_splc'] = df_nodups['job_title_no_stpw'].str.replace(r'[^A-Za-z0-9 ]+', '', regex=True)
df_nodups['job_title_no_splc'] = df_nodups['job_title_no_splc'].str.replace(r'\d+','') 
df_nodups.head()

,id,job_title,location,connection,fit,job_title_no_stpw,job_title_no_splc
0,1,2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional,"Houston, Texas",85,NaN,2019 c.t. bauer college business graduate (magna cum laude) aspiring human resources professional,ct bauer college business graduate magna cum laude aspiring human resources professional
1,2,Native English Teacher at English Program in Korea,Kanada,500+,NaN,native english teacher english program korea,native english teacher english program korea
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN,aspiring human resources professional,aspiring human resources professional
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN,people development coordinator ryan,people development coordinator ryan
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN,advisory board member celal bayar university,advisory board member celal bayar university


In [9]:
lemmer = nltk.stem.WordNetLemmatizer()

def lem(words):
    return " ".join([lemmer.lemmatize(word,'v') for word in words.split()])

df_nodups['job_lemmatized'] =  df_nodups.job_title_no_splc.apply(lem)

df_nodups.head()

,id,job_title,location,connection,fit,job_title_no_stpw,job_title_no_splc,job_lemmatized
0,1,2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional,"Houston, Texas",85,NaN,2019 c.t. bauer college business graduate (magna cum laude) aspiring human resources professional,ct bauer college business graduate magna cum laude aspiring human resources professional,ct bauer college business graduate magna cum laude aspire human resources professional
1,2,Native English Teacher at English Program in Korea,Kanada,500+,NaN,native english teacher english program korea,native english teacher english program korea,native english teacher english program korea
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN,aspiring human resources professional,aspiring human resources professional,aspire human resources professional
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN,people development coordinator ryan,people development coordinator ryan,people development coordinator ryan
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN,advisory board member celal bayar university,advisory board member celal bayar university,advisory board member celal bayar university


In [10]:
# Convert job_title column into a list
job_title_list = list(df_nodups['job_lemmatized'])                    

# Vectorize job_title_list
vectors = TfidfVectorizer()                                 
matrix = vectors.fit_transform(job_title_list)
feature_names = vectors.get_feature_names()  
print("Number of unique features: ", len(feature_names))   
print(feature_names)

Number of unique features:  175
['administration', 'administrative', 'admissions', 'advisory', 'always', 'america', 'an', 'analyst', 'analytics', 'army', 'arts', 'aspire', 'assistant', 'atlanta', 'bachelor', 'bauer', 'bayar', 'beach', 'beneteau', 'biology', 'board', 'brand', 'buckhead', 'business', 'care', 'celal', 'center', 'chapman', 'college', 'communications', 'community', 'compensation', 'conflict', 'coordinator', 'create', 'csr', 'ct', 'cum', 'customer', 'data', 'delphi', 'development', 'director', 'employment', 'endemol', 'energetic', 'energy', 'engage', 'engie', 'engineer', 'english', 'entrylevel', 'environment', 'environmental', 'excellence', 'executive', 'experience', 'ey', 'generalist', 'gi', 'gp', 'graduate', 'groupe', 'guard', 'hardware', 'heil', 'help', 'houston', 'human', 'humber', 'illinois', 'inc', 'inclusive', 'indiana', 'information', 'intelligence', 'intercontinental', 'international', 'internship', 'japan', 'junior', 'kokomo', 'korea', 'lab', 'laude', 'lead', 'lead

In [11]:
vectors.vocabulary_

{'ct': 36,
 'bauer': 15,
 'college': 28,
 'business': 23,
 'graduate': 61,
 'magna': 94,
 'cum': 37,
 'laude': 84,
 'aspire': 11,
 'human': 68,
 'resources': 134,
 'professional': 125,
 'native': 105,
 'english': 50,
 'teacher': 159,
 'program': 127,
 'korea': 82,
 'people': 120,
 'development': 41,
 'coordinator': 33,
 'ryan': 138,
 'advisory': 3,
 'board': 20,
 'member': 102,
 'celal': 25,
 'bayar': 16,
 'university': 166,
 'specialist': 149,
 'student': 153,
 'humber': 69,
 'generalist': 58,
 'senior': 143,
 'seek': 142,
 'information': 74,
 'system': 157,
 'position': 123,
 'chapman': 27,
 'svp': 156,
 'market': 99,
 'communications': 29,
 'csr': 35,
 'officer': 110,
 'engie': 48,
 'houston': 67,
 'the': 161,
 'woodlands': 172,
 'energy': 46,
 'gp': 60,
 'sp': 148,
 'intercontinental': 76,
 'buckhead': 22,
 'atlanta': 13,
 'management': 96,
 'internship': 78,
 'opportunities': 113,
 'experience': 56,
 'retail': 135,
 'manager': 98,
 'staff': 151,
 'recruit': 129,
 'luxottica': 93,


In [12]:
# Convert job titles into arrays
tfidf_vector = matrix.toarray()                                  
print("Shape of Tfidf vector: ", tfidf_vector.shape) 
print(tfidf_vector)

Shape of Tfidf vector:  (54, 175)
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.46104593 0.         0.         ... 0.         0.         0.        ]]


In [13]:
print(tfidf_vector[0])

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.19276841
 0.         0.         0.         0.3511579  0.         0.
 0.         0.         0.         0.         0.         0.26173515
 0.         0.         0.         0.         0.31815462 0.
 0.         0.         0.         0.         0.         0.
 0.3511579  0.3511579  0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.31815462 0.         0.         0.         0.
 0.         0.         0.11589292 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.3511579  0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.3511579  0.
 0.         0.         0.         0.    

In [14]:
# search query
search = "seeking human resources"


In [15]:
lemmer = nltk.stem.WordNetLemmatizer()
lem_search = " ".join([lemmer.lemmatize(word,'v') for word in search.split()])
print(lem_search)

seek human resources


In [16]:
# Convert search phrase into a vector
sv = vectors.transform([lem_search])                    
search_vector = sv.toarray()
print("Shape of search phrase vector:", search_vector.shape)
print(search_vector)

Shape of search phrase vector: (1, 175)
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.4319816  0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.


In [17]:
cosine_sim = cosine_similarity(tfidf_vector, search_vector)

df_nodups['tfidf_sim_score'] = cosine_sim
df_nodups.sort_values(by ='tfidf_sim_score', ascending = False).head()

,id,job_title,location,connection,fit,job_title_no_stpw,job_title_no_splc,job_lemmatized,tfidf_sim_score
98,99,Seeking Human Resources Position,"Las Vegas, Nevada Area",48,NaN,seeking human resources position,seeking human resources position,seek human resources position,0.696263
27,28,Seeking Human Resources Opportunities,"Chicago, Illinois",390,NaN,seeking human resources opportunities,seeking human resources opportunities,seek human resources opportunities,0.673137
72,73,"Aspiring Human Resources Manager, seeking internship in Human Resources.","Houston, Texas Area",7,NaN,"aspiring human resources manager, seeking internship human resources.",aspiring human resources manager seeking internship human resources,aspire human resources manager seek internship human resources,0.624745
9,10,Seeking Human Resources Human Resources Information System and Generalist Positions,Greater Philadelphia Area,500+,NaN,seeking human resources human resources information system generalist positions,seeking human resources human resources information system generalist positions,seek human resources human resources information system generalist position,0.517021
26,27,Aspiring Human Resources Management student seeking an internship,"Houston, Texas Area",500+,NaN,aspiring human resources management student seeking internship,aspiring human resources management student seeking internship,aspire human resources management student seek internship,0.459454


In [ ]:
#BERT

In [54]:
bert_model = SentenceTransformer('all-MiniLM-L6-v2')

In [73]:
X = df_nodups[['job_lemmatized']]
type(X)
X.shape

(54, 1)

In [78]:
# Convert job titles into BERT embedded vectors
bert_embeddings = bert_model.encode(job_title_list)
bert_embeddings.shape

(54, 384)

In [64]:
bert_embeddings[0].shape

(384,)

In [88]:
bert_embeddings_new1 = bert_embeddings[0].reshape(384, 1)
bert_embeddings_new1

array([[-8.84988308e-02],
       [ 3.59814912e-02],
       [ 4.53154603e-03],
       [ 5.32625895e-03],
       [-2.90048905e-02],
       [ 1.54956831e-02],
       [-6.72541186e-02],
       [-1.55005837e-04],
       [-1.39666274e-02],
       [ 1.65754836e-02],
       [-2.25748401e-02],
       [-3.46604809e-02],
       [-9.12403688e-02],
       [-5.05309924e-02],
       [-7.34344870e-02],
       [ 2.75597479e-02],
       [-7.89302308e-03],
       [-6.26581982e-02],
       [ 2.05161963e-02],
       [ 1.13258855e-02],
       [ 5.24467342e-02],
       [ 2.96305586e-02],
       [-5.47177382e-02],
       [-7.93486163e-02],
       [-1.25118103e-02],
       [-2.76623517e-02],
       [-9.26052779e-03],
       [-1.89929623e-02],
       [ 4.08062786e-02],
       [-2.02714689e-02],
       [-7.29745114e-03],
       [ 8.61455780e-03],
       [ 2.61284634e-02],
       [ 4.92619909e-02],
       [ 3.03808078e-02],
       [ 8.53083655e-02],
       [-8.86077469e-04],
       [ 8.27061944e-03],
       [ 1.5

In [65]:
# Convert search phrase into a BERT embedded vector
bert_search_embedding = bert_model.encode(lem_search)
bert_search_embedding.shape

(384,)

In [89]:
bert_embeddings_new2 = bert_search_embedding.reshape(384, 1)
bert_embeddings_new2

array([[-1.44166742e-02],
       [-7.56161055e-03],
       [-1.99380927e-02],
       [ 5.67318052e-02],
       [ 3.72525789e-02],
       [ 2.96661220e-02],
       [ 7.76815042e-02],
       [-5.78508712e-02],
       [-8.49030241e-02],
       [-2.73208208e-02],
       [ 2.01399811e-02],
       [-5.51739261e-02],
       [-6.36117980e-02],
       [ 1.32392971e-02],
       [ 2.10088287e-02],
       [ 1.46055957e-02],
       [ 1.13837924e-02],
       [ 7.89884012e-03],
       [ 5.99905103e-02],
       [-5.02241328e-02],
       [-5.70580177e-03],
       [ 4.55340818e-02],
       [-1.50071708e-02],
       [-1.15072303e-01],
       [-3.22948173e-02],
       [-1.68864313e-03],
       [-5.81313716e-03],
       [-3.18347290e-02],
       [ 6.58090487e-02],
       [ 1.97670478e-02],
       [ 1.14054769e-01],
       [-2.14311443e-02],
       [ 2.77212635e-02],
       [ 2.80831922e-02],
       [ 7.12781772e-02],
       [ 9.53591168e-02],
       [ 1.22220386e-02],
       [ 7.11557455e-03],
       [ 2.5

In [104]:
bert_cosine_similarity = []
for i in range(0, len(df_nodups)):
    bert_embeddings_new1 = bert_embeddings[i].reshape(384, 1)
    cosine_sim1 = cosine_similarity(bert_embeddings_new1, bert_embeddings_new2)
    bert_cosine_similarity.append(cosine_sim1)

df_nodups['BERT_sim_score'] = bert_cosine_similarity
df_nodups.head(1)

id  \
0   1   

                                                                                                  job_title  \
0  2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional   

         location connection  fit  \
0  Houston, Texas         85  NaN   

                                                                                   job_title_no_stpw  \
0  2019 c.t. bauer college business graduate (magna cum laude) aspiring human resources professional   

                                                                           job_title_no_splc  \
0   ct bauer college business graduate magna cum laude aspiring human resources professional   

                                                                           job_lemmatized  \
0  ct bauer college business graduate magna cum laude aspire human resources professional   

   tfidf_sim_score  \
0         0.100127   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [66]:
# Calculate cosine similarity between job title and search phrase vectors
bert_cosine_similarity = []
for i in range(0, len(df_nodups)):
    cos_sim = cosine_similarity(bert_embeddings[i], bert_search_embedding)
    bert_cosine_similarity.append(cos_sim)
    
#Add BERT_cosine_similarity column 
df_nodups['BERT_sim_score'] = bert_cosine_similarity
df_nodups.head()

ValueError: Expected 2D array, got 1D array instead:
array=[-6.98390380e-02  1.24659706e-02  5.65888546e-02  8.74389056e-03
 -3.18647511e-02  1.73637476e-02 -1.42417988e-02  2.68916674e-02
 -6.06675819e-03  4.80077900e-02  8.94897524e-03 -2.63810903e-02
 -7.94668794e-02 -5.07301055e-02 -1.80571042e-02  2.26087719e-02
 -1.19372094e-02 -1.83248334e-02 -6.42225565e-03 -7.51109570e-02
  6.68934956e-02  3.56645137e-02 -8.97612143e-03 -3.66832241e-02
 -4.61238483e-03 -3.09193805e-02  5.04971407e-02 -4.80032228e-02
  4.33549397e-02 -3.38911265e-02 -1.83362775e-02  8.51865299e-03
  6.93471506e-02  6.74879476e-02  6.42817989e-02  8.27020183e-02
 -1.20082954e-02 -3.02452091e-02  7.27669820e-02  1.31248524e-02
 -6.48469850e-02 -9.47883949e-02  5.78498468e-02 -9.36424546e-03
  3.00186034e-02 -6.05543293e-02 -2.31326148e-02 -8.06075037e-02
  1.95201952e-02  8.58491566e-03 -1.13201654e-02 -5.28009310e-02
 -1.12996427e-02  7.33299472e-04 -1.08096022e-02  4.99129109e-02
 -1.08868256e-02  6.55907691e-0